In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from get_data import *
from dataloader import *
from test_function import *
from NN_classes import TCN
import logging
import os



device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.set_default_dtype(torch.float64)

In [ ]:
# Save the model
# path = f"Ventil_trained_NNs\lstm_ws0.pth"
# #
# torch.save(model.state_dict(), path)

# Load the model and test it on the test data
path = r"Ventil_trained_NNs\tcn_0.pth"

params =                          {
                            "experiment_number" : 4,
                            "window_size" : 50,
                            "epochs" : 50,
                            "learning_rate" : 0.001,
                            "part_of_data" : 10, 
                            "percentage_of_data" : 0.7,
                            "batch_size" : 20,
                            "future" : 1,
                            "drop_half_timesteps" : True,
                            "cut_off_timesteps" : 0,

                            "input_channels" : 3,
                            "output" : 2,
                            "n_hidden" : 5,
                            "levels" : 4,
                            "kernel_size" : 5,
                            "dropout" : 0

                        }



input_data1, PSW_max = get_data(path = "data\save_data_test_revised.csv", 
                        timesteps_from_data=0, 
                        skip_steps_start = 0,
                        skip_steps_end = 0, 
                        drop_half_timesteps = params["drop_half_timesteps"],
                        normalise_s_w="minmax",
                        rescale_p=False,
                        num_inits=params["part_of_data"])

input_data2, PSW_max = get_data(path = "data\save_data_test5.csv", 
                        timesteps_from_data=0, 
                        skip_steps_start = 0,
                        skip_steps_end = 0, 
                        drop_half_timesteps = params["drop_half_timesteps"],
                        normalise_s_w="minmax",
                        rescale_p=False,
                        num_inits=params["part_of_data"])

input_data3, PSW_max = get_data(path = "data\Testruns_from_trajectory_generator_t2_t6_revised.csv", 
                        timesteps_from_data=0, 
                        skip_steps_start = 0,
                        skip_steps_end = 0, 
                        drop_half_timesteps = params["drop_half_timesteps"],
                        normalise_s_w="minmax",
                        rescale_p=False,
                        num_inits=params["part_of_data"])
                        

input_data = torch.cat((input_data1, input_data2, input_data3))
input_data=input_data1

np.random.seed(1234)
print("input_data size", input_data.size())
num_of_inits_train = int(len(input_data)*params["percentage_of_data"])
train_inits = np.random.choice(np.arange(len(input_data)),num_of_inits_train,replace=False)
test_inits = np.array([x for x in range(len(input_data)) if x not in train_inits])
np.random.shuffle(train_inits)
np.random.shuffle(test_inits)
test_data = input_data[test_inits,:,:]
np.random.seed()

# Initialize the LSTM model

input_channels = params["input_channels"]
output = params["output"]
num_channels = [params["n_hidden"]] * params["levels"]
kernel_size = params["kernel_size"]
dropout = params["dropout"]

model = TCN(input_channels, output, num_channels, kernel_size=kernel_size, dropout=dropout)

model.load_state_dict(torch.load(path, map_location=torch.device(device)))

train_data = input_data[train_inits,:,:]
#%matplotlib qt 
#%matplotlib inline 

#test_loss, test_loss_deriv, total_loss, physloss
test_loss, test_loss_deriv, total_loss = test(train_data, model, model_type = "tcn", window_size=params["window_size"],
                                                         display_plots=True, num_of_inits = 3, set_rand_seed=False, physics_rescaling = PSW_max, additional_data=None)
#test_loss, test_loss_deriv, total_loss = test(train_data, model, steps=input_data.size(dim=1), ws=params["window_size"], plot_opt=True , n = 1,  test_inits=len(test_data), rand=False, PSW_max=0)

#[350000.0, 0.0006, 1.7, 0.0, 0.0, -1.7]

In [ ]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


In [ ]:
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
       # self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
       # self.dropout2 = nn.Dropout(dropout)

        # self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
        #                          self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1,
                                 self.conv2, self.chomp2, self.relu2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

In [ ]:

class TemporalConvNet(nn.Module):


    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [ ]:
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])

In [ ]:

path = r"Ventil_trained_NNs\tcn_0.pth"

params =                 {
                            "window_size" : 50,
                            "learning_rate" : 0.001,
                            "part_of_data" : 10, 
                            "percentage_of_data" : 0.7,
                            "batch_size" : 20,
                            "drop_half_timesteps" : True,
                            "cut_off_timesteps" : 0,

                            "input_channels" : 3,
                            "output" : 2,
                            "n_hidden" : 5,
                            "levels" : 1,
                            "kernel_size" : 7,
                            "dropout" : 0

                        }



input_data1, PSW_max = get_data(path = "data\save_data_test_revised.csv", 
                        timesteps_from_data=0, 
                        skip_steps_start = 0,
                        skip_steps_end = 0, 
                        drop_half_timesteps = params["drop_half_timesteps"],
                        normalise_s_w="minmax",
                        rescale_p=False,
                        num_inits=params["part_of_data"])

input_data=input_data1

np.random.seed(1234)
print("input_data size", input_data.size())
num_of_inits_train = int(len(input_data)*params["percentage_of_data"])
train_inits = np.random.choice(np.arange(len(input_data)),num_of_inits_train,replace=False)
test_inits = np.array([x for x in range(len(input_data)) if x not in train_inits])
np.random.shuffle(train_inits)
np.random.shuffle(test_inits)
test_data = input_data[test_inits,:,:]
np.random.seed()
train_data = input_data[train_inits,:,:]


input_channels = params["input_channels"]
output = params["output"]
num_channels = [params["n_hidden"]] * params["levels"]
kernel_size = params["kernel_size"]
dropout = params["dropout"]

model = TCN(input_channels, output, num_channels, kernel_size=kernel_size, dropout=dropout)





In [ ]:
import torch
from torchviz import make_dot
from IPython.display import Image

y = model(input_data1[0:1].transpose(1,2))
print(input_data1[0:1].size())
dot = make_dot(y, params=dict(model.named_parameters()),  show_attrs=True, show_saved=True)   # .render("model_graph", format="png")

#print(list(model.parameters()))

#calc 
erg=0
for a in list(model.parameters()):
    x = 1 
    for y in list(a.size()):
        x = x*y
    erg += x

print("anzahl der variablen" , erg)
# Remove autograd related nodes
#dot.attr(rankdir='LR')
dot.node_attr.update(style='filled')

# for node in dot.body:
#     if 'Backward' in node:
#         dot.body.remove(node)


dot
# Save the graph to a file
# dot.format = 'png'
# dot.render('model_graph')

# Display the graph in the Jupyter notebook
#Image(filename='model_graph.png')